In [69]:
import numpy as np

data_a3b = np.load('Bi_a3b.npy')[()]
data_ab = np.load('Bi_ab.npy')[()]


In [70]:
from ase.build import bulk
from ase.db import connect
from ase.visualize import view

def get_alloy(name, a):

    if '2' in name:
        symbols = name.split('2')[:2]
        system = 'L10'
        atoms = bulk(symbols[0], 'fcc', a=float(a['a']), cubic=True)
        atoms.set_cell([float(a['a']), float(a['a']), float(a['c'])], scale_atoms=True)
        atoms[1].symbol = symbols[1]
        atoms[2].symbol = symbols[1]

    elif '3' in name:
        symbols = name.split('3')
        system = 'L12'
        atoms = bulk(symbols[0], 'fcc', a=float(a), cubic=True)
        atoms[1].symbol = symbols[1]

    return atoms, system

db = connect('bulk.db')

for k, v in data_a3b.items():
    atoms, system = get_alloy(k, v)

    keys = {
        'SBS_symbol': system,
    }    

    keys['a'] = float(v)
    keys['metal'] = k
    
    db.write(atoms, key_value_pairs=keys)
    
for k, v in data_ab.items():
    atoms, system = get_alloy(k, v)

    keys = {
        'SBS_symbol': system,
    }    
    keys['a'] = float(v['a'])
    keys['c'] = float(v['c'])
    keys['metal'] = k
    
    db.write(atoms, key_value_pairs=keys)

In [71]:
from catkit.surface import SlabGenerator
from ase.db import connect
import numpy as np
from ase.data import atomic_numbers as an
from ase.data import vdw_radii as vradii
from ase.data import covalent_radii as cradii
from numpy.linalg import norm
from ase import Atom
from catkit import utils

adsorbates = ['H', 'C', 'N', 'O', 'S']
pol_metal = ['Fe', 'Ni', 'Co', 'Mn']

# Change any of the desired keywords here.
parameters = {
    'mode': 'relax',
    'opt_algorithm': 'bfgs',
    'xc': 'BEEF-vdW',
    'kpts': (6, 6, 1),
    'pw': 500,
    'dw': 5000,
    'sigma': 0.15,
    'nbands': -10,
    'fmax': 0.05,
    'beefensemble': True,
    'nosym': True,
    'outdir': '.',
    'calcstress': True,
    'dipole': {'status': True},
    'output': {
        'removesave': True,
        'avoidio': False,
        'removewf': True,
        'wf_collect': False,
    },
    'convergence': {
            'energy': 1e-5 * 13.6,
            'mixing': 0.3,
            'nmix': 10,
            'mix': 4,
            'maxsteps': 2000,
            'diag': 'david'
    },
}

keys = {
    'facet': '(1, 1, 1)',
    'layer': 3,
    'vacuum': 10,
}

dbm = connect('bulk.db')
db = connect('slab.db')


for d in dbm.select():
    bulk = d.toatoms()
    kvp = d.key_value_pairs

    keys.update(kvp)

    metals = list(set(bulk.get_chemical_symbols()))

    gen = SlabGenerator(
        bulk,
        miller_index=(1, 1, 1),
        layers=3,
        fixed=2,
        vacuum=10,
    )

    atoms = gen.get_slab()
         
    keys['metal'] = kvp['metal']
    keys['bulk_id'] = d.id

    atoms.wrap()
    parameters['spinpol'] = False
    if metals[0] in pol_metal or metals[1] in pol_metal:
        parameters['spinpol'] = True

        atoms.set_initial_magnetic_moments(
        [2.0 if atom.symbol in pol_metal else 0 for atom in atoms]
                    )
    db.write(atoms, key_value_pairs=keys, data=parameters)


    sites, vslab = gen.adsorption_sites(
        atoms,
        return_proxy=True
    )

    pos = vslab.positions
    num = vslab.get_atomic_numbers()

    for k, v in sites.items():
        coordinates, points, neighbors = v

        for i, s in enumerate(coordinates):

            n = num[points[i]]

            for ads in adsorbates:

                rA = cradii[an[ads]]
                slab = atoms.copy()

                # Top site
                if isinstance(points[i], np.int64):
                    rM = cradii[n]
                    slab += Atom(ads, coordinates[i] + [0, 0, rM + rA])

                # Bridge site
                elif len(points[i]) == 2:
                    slab += Atom(ads, coordinates[i] + [0, 0, rA + 0.5])

                # Hollow site
                elif len(points[i]) == 3:
                    inter = utils.trilaterate(pos[points[i]], vradii[n])
                    if not np.isnan(inter).any():
                        slab += Atom(ads, inter + [0, 0, rA])
                    else:
                        slab += Atom(ads, coordinates[i] + [0, 0, rA + 0.2])

                slab.rattle(1e-3)
                slab.wrap()
                parameters['spinpol'] = False
                if metals[0] in pol_metal or metals[1] in pol_metal:
                    parameters['spinpol'] = True

                    slab.set_initial_magnetic_moments(
                        [2.0 if atom.symbol in pol_metal else 0 for atom in slab]
                    )

                keys['adsorbate'] = ads
                keys['site'] = '{}'.format(k)
                keys['site_id'] = int(i)
                keys['bulk_id'] = d.id
                keys['metal'] = kvp['metal']

                db.write(slab, key_value_pairs=keys, data=parameters)



/Users/osmanmamun/.local/lib/python3.6/site-packages/CatKit-0.1.0-py3.6.egg/catkit/utils.py:32: RuntimeWarning: invalid value encountered in sqrt
